In [1]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import os

# os.environ["OPENAI_API_KEY"] = "secret-api-key"
os.environ["OPENAI_API_KEY"] = "sk-"

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(
    model_name="text-davinci-003",
    temperature=0.0,
)

result = llm("自己紹介求！")
print(result)




はじめまして！
私は、Taroと申します。
大学では、経済学を専攻しています。
趣味は、スポーツ観戦や旅行などです。
特に、サッカーが大好きで、日本代表の試合を観戦するのが大好きです。
今後ともよろしくお願いします！


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# LangChain で Chat Completions API を使う。
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="俺はジョンだ！"),
    AIMessage(content="こんにちはジョンさん、どのように手伝えますかあ？"),
    HumanMessage(content="私の名前がわかるかな、ははは"),
]

result = chat(messages)
print(result)
print(result.content)


content='はい、先ほど「俺はジョンだ！」とおっしゃいましたので、お名前はジョンさんですね。どのようなお手伝いができますか？'
はい、先ほど「俺はジョンだ！」とおっしゃいましたので、お名前はジョンさんですね。どのようなお手伝いができますか？


In [7]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# 標準出力にストリーム出力する。
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

messages = [HumanMessage(content="自己紹介求む！")]
result = chat(messages)


こんにちは！私はAIのチャットボットです。私の名前はOpenAI Assistantです。私は様々なトピックについて質問に答えたり、会話をすることができます。私は自然言語処理技術を使用しており、ユーザーの質問や要求に対して最善の回答を提供するように設計されています。どのようにお手伝いできますか？

In [8]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを考えてください。

料理名: {dish}
"""

# プログラムで文字列の一部を置き換えているだけで、
# 内部で LLM を呼び出すことはしていない。
prompt = PromptTemplate(
    input_variables=["dish"],
    template=template,
)

result = prompt.format(
    dish = "カレーー",
)

print(result)



以下の料理のレシピを考えてください。

料理名: カレーー



In [10]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

# ChatPromptTemplate は PromptTemplate を Chat Completions API の形式に対応させたもの。
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
    HumanMessagePromptTemplate.from_template("以下の料理のレシピを考えてください。\n\n料理名: {dish}"),
])

messages = chat_prompt.format_prompt(
    country="中国",
    dish="肉じゃが",
).to_messages()

print(messages)


[SystemMessage(content='あなたは中国料理のプロフェッショナルです。'), HumanMessage(content='以下の料理のレシピを考えてください。\n\n料理名: 肉じゃが')]


In [11]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = parser.get_format_instructions()

print(format_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "ingredients of the dish", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "steps to make the dish", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```


In [12]:
template = """
料理のレシピを考えてください！！

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": format_instructions}
)

formatted_prompt = prompt.format(
    dish = "カレーー",
)

print(formatted_prompt)



料理のレシピを考えてください！！

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "ingredients of the dish", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "steps to make the dish", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```

料理名: カレーー



In [13]:
# PromptTemplate と Output parser を合わせる。
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
)
messages = [HumanMessage(content=formatted_prompt)]
output = chat(messages)
print(output.content)


{
  "ingredients": [
    "玉ねぎ",
    "にんじん",
    "じゃがいも",
    "牛肉",
    "カレールー",
    "水"
  ],
  "steps": [
    "玉ねぎ、にんじん、じゃがいもを切ります。",
    "牛肉を炒めます。",
    "野菜を加えて炒めます。",
    "水を加えて煮込みます。",
    "カレールーを加えて溶かします。",
    "煮込んで完成です。"
  ]
}


In [14]:
# Pydantic に変換して使う。
recipe = parser.parse(output.content)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '牛肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切ります。', '牛肉を炒めます。', '野菜を加えて炒めます。', '水を加えて煮込みます。', 'カレールーを加えて溶かします。', '煮込んで完成です。']


# LLMChain

Prompt Template, Language model, Output Parser を繋ぐ


In [15]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

# Output Parser
output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """
料理のレシピを考えてください！！

{format_instructions}

料理名: {dish}
"""

# Prompt Template
prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# Language model
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
)


In [17]:
from langchain import LLMChain

chain = LLMChain(
    prompt = prompt,
    llm = chat,
    output_parser = output_parser,
)

recipe = chain.run(
    dish = "カレーーー",
)

print(type(recipe))
print(recipe)


<class '__main__.Recipe'>
ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '牛肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切ります。', '牛肉を炒めます。', '野菜を加えて炒めます。', '水を加えて煮込みます。', 'カレールーを加えて溶かします。', '煮込んで完成です。']


## SimpleSequentialChain の例！

In [18]:
chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.0,
)

cot_template = """
以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template = cot_template,
)

# 1st chain:
# Zero-shot CoT でステップバイステップで考えさせる。
cot_chain = LLMChain(
    llm = chat,
    prompt = cot_prompt,
)

summarize_template = """
以下の文章を結論だけ一言に要約してください。

{input}
"""

summarize_prompt = PromptTemplate(
    input_variables = ["input"],
    template = summarize_template,
)

# 2st chain:
# 出力内容を要約する。
summarize_chain = LLMChain(
    llm = chat,
    prompt = summarize_prompt,
)

# 2つを繋げた Chain を作成する。

from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(
    chains = [cot_chain, summarize_chain],
)

result = cot_summarize_chain(
    "スーパーで10個のリンゴを買いました。友達に2つ、妹に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？",
)

print(result["output"])


残りのリンゴは10個です。


# Memory

In [19]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(
    model_name = "gpt-4",
    temperature = 0.0,
)

conversation = ConversationChain(
    llm = chat,
    memory = ConversationBufferMemory(),
)

while True:
    user_message = input("You: ")
    ai_message = conversation.run(input=user_message)
    print(f"AI: {ai_message}")


You: pien
AI: It seems like you've typed "pien," which doesn't form a clear question or statement in English. If it's a typo, could you please correct it? If it's a term in another language, could you please provide more context or specify the language? I'm here to help!
You: ぴえん
AI: Ah, you're using Japanese! "ぴえん" is a slang term in Japanese that's often used on the internet. It's an onomatopoeic word that represents a crying sound, similar to "boo hoo" in English. It's often used to express sadness or disappointment.


KeyboardInterrupt: ignored

In [20]:
# Chat Completions API へのリクエストの内容をログ出力させたい！
import logging
logging.getLogger('openai').setLevel(logging.DEBUG)
